# template_webscraping

## Load Libraries

In [79]:
# Selenium
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
# Reqeusts
import requests
# Other tools
from bs4 import BeautifulSoup
from datetime import datetime
from datetime import timedelta
from datetime import date
from datetime import datetime
from datetime import timedelta
from datetime import date
import csv
import pandas as pd
import time
import json
import fnmatch
import os
import tabula
from tabula.io import read_pdf


## Functions Using Requests

In [80]:
def requests_get_item(url, item):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}
    if item == "html":       
        page = requests.get(url, headers = headers)
        soup = BeautifulSoup(page.text, 'html.parser')
        return(soup)
    
    elif item == "pdf":
        page = requests.get(url, headers = headers, stream=True)
        return(page)
    else:
        print("Valid Item Not Selected")

def download_pdfs(ls_pdf_urls, download_path, file_name):
    counter = 0 
    for pdf_url in ls_pdf_urls:
        counter = counter +1
        g = requests_get_item(pdf_url, "pdf")
        with open(f'{download_path}{file_name}_{counter}.pdf', 'wb') as sav:
            for chunk in g.iter_content(chunk_size=1000000):
                sav.write(chunk)
        print(f"download number: {counter}")
                
                
def convert_pdf_to_csv(pdf_directory, csv_directory):
    directory = fr'{pdf_directory}'
    directory_output = fr'{csv_directory}'
    count = 0
    for file in os.listdir(directory):        
        if file.endswith(".pdf"):
            count = count + 1 
            print(f'{directory}{file}: Conversion {count}')
            tabula.convert_into(f'{directory}{file}', f'{directory_output}{count}.csv', output_format="csv", pages='all')
    
    
    
    

## Functions Using Selenium

In [81]:
# Define important paths
executable_path = "./aadit_chromedriver"
download_path = f"..data/handmade/salsbury/pdfs/"

# Define Routes (usually in xpaths)

def browser_instance(executable_path, headless, download_path):
    
    
    chrome_options =  Options()
    if headless:
        chrome_options.add_argument("--headless")   
    chrome_options.add_experimental_option("prefs", {
            "download.default_directory": download_path})    
    driver = webdriver.Chrome(executable_path=executable_path, options = chrome_options)
    return driver  

def signIn(driver, url, signin_field, username_field, password_field):  
    driver.get(url)
    sign_in = driver.find_element_by_xpath(signin_field)
    sign_in.click()
    email_field = driver.find_element_by_xpath(username_field)
    password_field = driver.find_element_by_xpath(password_field)
    email_field.send_keys(USERNAME)
    password_field.send_keys(PASSWORD + Keys.RETURN)


## Useful Selenium Concepts
### Waiting until element is clickable
button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, download_all_field)))
button.click()

### Try and Expect
try:
    attempt to do something
except:
    do this if the try doesn't work out
    


## General Functions

In [82]:
def write_json_data(data_out, database_name, file_path):
        #writing data to json
        data_out = data_out
        database_name = database_name
        file = f'{file_path}/{database_name}'
        with open(file, 'w') as outfile:
            json.dump(data_out, outfile)
            
def open_json_data(file_path):    
    with open(file_path) as f:
        data = json.load(f)
    return data

In [83]:
# date must be in following format: 2021-08-12
def get_date(start_date, end_date):
    date_list = []
    # end_date as current data
    end_date = date.today()
    # add 60 days to start date, save that as first el in list
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    start_date = start_date + timedelta(days=60)
    start_date = (start_date.date())
    
    date_list.append(str(start_date))
    while (start_date + timedelta(days=60)) < end_date:
        start_date = start_date + timedelta(days=120)
        string_start_date = str(start_date)
        
        if (start_date + timedelta(days=60)) > end_date: 
            start_date = end_date
        date_list.append(str(start_date))
    
        
    return(date_list)
        # while start + 60 is less than today, keep adding a new date that's +120 days
        # when it's greater thant today's date, break out of the lop
    
    # next date will be 120 days after the date we just returned
    # while loop intil we hit a date that's within 60 days of today's end_date
    
dates = get_date('2019-01-03', 'end_date')
print(dates)

['2019-03-04', '2019-07-02', '2019-10-30', '2020-02-27', '2020-06-26', '2020-10-24', '2021-02-21', '2021-06-21', '2021-10-12']


In [104]:
driver = browser_instance(executable_path, False, download_path)
driver.get("https://www.salisbury.edu/police/clery-compliance/daily-crime-and-incident-log.aspx")

In [ ]:


results = []
for date in dates[0:1]:
#     print(date)
    driver.get("https://www.salisbury.edu/police/clery-compliance/daily-crime-and-incident-log.aspx")
    search_bar = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="dateSearch"]')))
    submit_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="dateSubmit"]')))
    
    print(date)
    print("-----")
    search_bar.send_keys(date)
    
    submit_button.click()
#     search_bar.send_keys(Keys.DELETE)
#     search_bar.send_keys(Keys.COMMAND, 'a')
#     search_bar.send_keys(Keys.BACKSPACE)
    table_finder = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="crimebeattable"]')))
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    print(soup.prettify())
#     table = soup.find(id="crimebeattable")
#     rows = table.find_all('tr')
#     for row in rows[1:]:
#       cells = row.find_all('td')
#       results.append([cell.text for cell in cells])
        
# with open('1.csv','w') as output_file:
#     csvfile = csv.writer(output_file)
#     csvfile.writerow(['case', 'date_time_location','date_reported', 'incident', 'disposition'])
#     csvfile.writerows(results)


In [160]:
### generate urls -
# url = https://www.salisbury.edu/_services/police/crimebeat?date=07%2F12%2F2017


base_url = 'https://www.salisbury.edu/_services/police/crimebeat?date='
results = []
counter = 0
counter2 = 0
for date in dates:
#     print(date)
    pass
    split_date = date.split('-')
    for num in split_date:
        year = (split_date[0])
        month = (split_date[1])
        day = (split_date[2])
#         print(year, month, year)
#         print('---')
#         print(url)
#         print('---')
        url = (f'{base_url}{month}%2F{day}%2F{year}')
#         print(url)
#         counter2 = counter2 + 1
#         print(counter2)
#         print('---')
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')
#         counter2 = counter2 + 1
#         print(counter2)
        results.append(soup.text)
#         print(results)
for result in results:
    
    counter = counter + 1
    print(counter)
#             print(type(result))
#             print('-----------')
    file = f'../data/handmade/salisbury/jsons/{counter}output.json'
    with open(file, 'w') as outfile:
        json.dump(result, outfile)
            
# print(results)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


In [ ]:

def write_json_data(data_out, database_name, file_path):
        #writing data to json
        data_out = data_out
        database_name = database_name
        file = f'{file_path}/{database_name}'
        with open(file, 'w') as outfile:
            json.dump(data_out, outfile)